# Transcript ETL Code

## Preamble

In [2]:
!pip install pandera

     ------------------------------------ 115.9/115.9 kB 754.9 kB/s eta 0:00:00
     ---------------------------------------- 10.6/10.6 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.4
    Uninstalling pandas-1.1.4:
      Successfully uninstalled pandas-1.1.4


In [1]:

from pandera.typing import DataFrame

import pandas
import json
from pathlib import Path



## Config

In [2]:
project_dir = Path("../../")
project_dir

WindowsPath('../..')

In [3]:
podcast_dir = project_dir / "data/podcasts/"
podcast_dir

WindowsPath('../../data/podcasts')

In [4]:
transcript_dir = project_dir / "data/transcripts/Google"
transcript_dir

WindowsPath('../../data/transcripts/Google')

In [5]:
ls {transcript_dir}

 Volume in drive H has no label.
 Volume Serial Number is 6684-7A77

 Directory of H:\wav2vec_pre\podcast-ai-lab\data\transcripts\Google

09-07-2022  21:58    <DIR>          .
09-07-2022  21:58    <DIR>          ..
09-07-2022  21:58           231,318 20220617 lex_ai_richard_wolff.json
09-07-2022  21:58           183,984 20220617 lex_ai_richard_wolff.vtt
09-07-2022  21:58           174,166 20220628 lex_ai_susan_cain.json
09-07-2022  21:58           141,336 20220628 lex_ai_susan_cain.vtt
09-07-2022  21:58           249,734 20220701 lex_ai_demis_hassabis.json
09-07-2022  21:58           168,142 20220701 lex_ai_demis_hassabis.vtt
               6 File(s)      1,148,680 bytes
               2 Dir(s)  172,918,071,296 bytes free


## Data Transforms

### Transcript to DataFrame

In [6]:
example_transcript_path = transcript_dir / "20220701 lex_ai_demis_hassabis.json"

In [7]:
def google_transcript_to_dataframe(
    path,
) -> DataFrame:
    """Parse Google Speech to Text API result .json -> pandas.DataFrame"""
    transcript_data = pandas.read_json(path)
    return transcript_data

In [8]:
def wav2vec_transcript_to_dataframe(
    path,
) -> DataFrame:
    """Parse Wav2Vec result .csv -> pandas.DataFrame"""
    transcript_data = pandas.read_csv(path)
    return transcript_data

In [9]:
def parse_google_transcript(
    path,
    episode_id: int = None,
) -> DataFrame:
    """Parse Google Speech to Text API result .json -> pandas.DataFrame"""
    transcript_data = pandas.read_json(path)
    #transcript_data["file_name"] = path.stem
    transcript_data["episode_id"] = episode_id
    return transcript_data

In [10]:
def clean_google_transcript(
    transcript_data,
) -> DataFrame:
    """Clean Google Speech to Text API result."""
    transcript_data = (
        transcript_data
        .rename(
            columns={
                "start": "chunk_start",
                "end": "chunk_end",
                "newpara": "new_paragraph"
            }
        )
    )
    transcript_data["new_paragraph"] = transcript_data["new_paragraph"].fillna(0.0).astype(bool)
    transcript_data["chunk_number"] = transcript_data.index
    # enumerate paragraphs
    transcript_data["paragraph_number"] = transcript_data["new_paragraph"].cumsum()
    transcript_data = transcript_data.drop(columns=["new_paragraph"])
    return transcript_data


In [11]:
def process_transcripts(
    transcript_dir,
):
    """Process all transcripts in a directory."""
    transcript_paths = [
        p for p in transcript_dir.glob("*.json")
    ]
    transcript_data = [
        clean_google_transcript(parse_google_transcript(path, episode_id)) for episode_id, path in enumerate(transcript_paths)
    ]
    episode_data = pandas.DataFrame(
        [
            {"episode_id": episode_id, "file_name": path.stem}
            for episode_id, path in enumerate(transcript_paths)
        ]
    )
    transcript_data = pandas.concat(transcript_data)
    transcript_data = transcript_data.set_index(["episode_id", "chunk_number"])
    episode_data = episode_data.set_index("episode_id")
    return transcript_data, episode_data

## Examples

TODO:
- chunk enumeration
- file id / name

In [12]:
transcript_data = clean_google_transcript(
    parse_google_transcript(
        transcript_dir / "20220701 lex_ai_demis_hassabis.json"
    )
)
transcript_data

,chunk_start,chunk_end,text,confidence,episode_id,chunk_number,paragraph_number
0,0.018,6.398,The following is a conversation with demouth c...,0.80,None,0,1
1,6.602,16.615,A company that has published and build some of...,0.88,None,1,1
2,16.828,30.299,All by itself to play the game of Go better th...,0.90,None,2,1
3,31.088,40.678,Thomas is widely considered to be one of the m...,0.80,None,3,2
4,40.873,50.563,This was truly an honor and a pleasure for me ...,0.88,None,4,2
...,...,...,...,...,...,...,...
921,8172.342,8184.166,Human beings in this giant puzzle of ours and ...,0.81,None,921,203
922,8184.954,8191.394,Thanks for listening to this conversation with...,0.81,None,922,204
923,8191.689,8195.580,And now let me leave you with some words from ...,0.91,None,923,205
924,8196.244,8202.197,Computer science is no more about computers an...,0.91,None,924,206


In [15]:
transcript_data.to_csv('transcript.csv')

In [15]:
transcript_data, episode_data = process_transcripts(
    transcript_dir=transcript_dir,
)

In [16]:
episode_data.to_

,file_name
episode_id,
0,20220617 lex_ai_richard_wolff
1,20220628 lex_ai_susan_cain
2,20220701 lex_ai_demis_hassabis


In [17]:
episode_data?

Type:        DataFrame
String form:
file_name
           episode_id
           0           <...> chard_wolff
           1               20220628 lex_ai_susan_cain
           2           20220701 lex_ai_demis_hassabis
Length:      3
File:        c:\users\acer\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py
Docstring:  
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, dataclass or list-like objects. If
    data is a dict, column order follows insertion-order. If a dict contains Series
    which have an index defined, it is aligned by its index.

    .. versionchanged:: 0.25.0
   

In [15]:
transcript_data

chunk_start  chunk_end  \
episode_id chunk_number                           
0          0                   0.018      6.398   
           1                   6.602     16.615   
           2                  16.828     30.299   
           3                  31.088     40.678   
           4                  40.873     50.563   
...                              ...        ...   
2          864              7495.120   7503.117   
           865              7503.708   7511.174   
           866              7511.783   7521.274   
           867              7522.478   7525.730   
           868              7526.960   7542.264   

                                                                      text  \
episode_id chunk_number                                                      
0          0             The following is a conversation with demouth c...   
           1             A company that has published and build some of...   
           2             All by itself to play the game of Go better th...   
           3             Thomas is widely considered to be one of the m...   
           4             This was truly an honor and a pleasure for me ...   
...                                                                    ...   
2          864           If you'll accept really strong emotions someti...   
           865           Highly sensitive people also process informati...   
           866           They tend to notice so these that others miss ...   
           867           Thank you for listening and hope to see you ne...   
           868                                                      Music.   

                         confidence  paragraph_number  
episode_id chunk_number                                
0          0                   0.80                 1  
           1                   0.88                 1  
           2                   0.90                 1  
           3                   0.80                 2  
           4                   0.88                 2  
...                             ...               ...  
2          864                 0.90               178  
           865                 0.91               178  
           866                 0.81               178  
           867                 0.85               179  
           868                  NaN               180  

[3037 rows x 5 columns]

In [16]:
def load_knowledge_base(
    knowledge_base_dir,
):
    """Load knowledge base."""
    transcript_data = pandas.read_parquet(knowledge_base_dir / "transcript_data.parquet")
    episode_data = pandas.read_parquet(knowledge_base_dir / "episode_data.parquet")

    return {
        "transcript_data": transcript_data, 
        "episode_data": episode_data,
    }

In [19]:
from engine import config

ModuleNotFoundError: No module named 'engine'

In [20]:
load_knowledge_base(
    knowledge_base_dir="../../data/knowledge_base/"
)

TypeError: unsupported operand type(s) for /: 'str' and 'str'